# Fine-tuning the modified model with our data

## Imports


In [1]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torchvision
import numpy as np
import torch
import argparse
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import torch.nn as nn
from collections import OrderedDict
import torch.optim as optim
import time
import copy
import torchvision.transforms as transforms
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import sys
sys.path.append('/content/drive/MyDrive/drive_folder')
from custom_dataset_loader import TaiChiDataset, ToTensor

## Helper functions

In [3]:
# converting all the images to tensors and then normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Our prediction model

In [36]:
def initialise_model(device):
  # Initialize model with the best available weights
  # create an instance of (e.g.) torchvision.models.segmentation.fcn_resnet50
  # and tell it to load pretrained weights
  weights = FCN_ResNet50_Weights.DEFAULT
  modified_model = fcn_resnet50(weights=weights)

  # we are feature extracting so we only need to compute weights for the new layer
  set_parameter_requires_grad(modified_model, True)

  # modify that model by removing its final layer and replacing with a 2D vector output at each pixel(?) (instead of 20 class logits)
  # instead of torch.Size([1, 21, 120, 240]) -> torch.Size([1, 2, 120, 240])
  modified_model.classifier[3] = nn.Sequential()
  modified_model.classifier[4] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
  # modified_model.aux_classifier[3] = nn.Sequential()
  # modified_model.aux_classifier[4] = nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
  # print(modified_model)

  # model to train() and load onto computation devicce
  modified_model.to(device)

  return modified_model

### Train function

In [14]:
# define train function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_aux=False):
  model.to(device)
  since = time.time()

  val_loss_history = []

  best_model_wts = copy.deepcopy(model.state_dict())

  for epoch in range(num_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            total_samples=0

            # Iterate over data.
            for i_batch, sample_batched in enumerate(dataloaders[phase]):
              batch_size = len(sample_batched['id'])
              total_samples+=batch_size

              inputs = sample_batched['image0']
              coords = sample_batched['coords']

              # coords = coords.view(batch_size,2,4096,1) # torch.Size([16, 1, 4096, 2])-> torch.Size([16, 2, 4096, 1])


              inputs = inputs.to(device).float() #torch.Size([16, 3, 120, 240])
              coords = coords.to(device) 
              
              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    outputs = outputs['out'].view(batch_size,1,-1,2) # torch.Size([16, 2, 120*240, 1])
                    # print(outputs.shape)
                    outputs = outputs[:, :, :4096, :]
                    loss = criterion(outputs, coords)
                    
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / total_samples
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # deep copy the model
            if epoch == 0:
              best_loss = epoch_loss
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_loss_history.append(epoch_loss)

            print()
            
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val loss: {:4f}\n'.format(best_loss))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model, val_loss_history




## Initializing Datasets and Dataloaders

In [ ]:
# Load dataset

TRAIN_DATA = "training_data_2023-01-10"
!unzip -d "$TRAIN_DATA"/ /content/drive/MyDrive/"$TRAIN_DATA".zip # unziping training data

dataset = TaiChiDataset(log_file=TRAIN_DATA+'/sample_ids.txt',
                        root_dir=TRAIN_DATA,
                        check=True,
                        transform=ToTensor()
                        )

In [12]:
def split_dataset(dataset, validation_split, batch_size, shuffle_dataset, random_seed):
  dataset_size = len(dataset)
  indices = list(range(dataset_size))
  split = int(np.floor(validation_split * dataset_size))
  if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
  train_indices, val_indices = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_indices)
  valid_sampler = SubsetRandomSampler(val_indices)
  
  # print(len(train_sampler))
  # print(len(valid_sampler))
  
  train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
  validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

  return train_loader, validation_loader

## Training & evaluating the model

In [45]:
# set computation device
device = torch.device('cuda')

# initialise model
modified_model = initialise_model(device)

# see what parameters will be tuned
params_to_update = modified_model.parameters()
print('Params to learn:')
for name, param in modified_model.named_parameters():
  if param.requires_grad:
      print(name)

Params to learn:
classifier.4.weight
classifier.4.bias


In [46]:
# Define parameters
validation_split = .2
shuffle_dataset = True
random_seed = 42
batch_size = 64
num_epochs = 30

# Get train and validation dataloaders
train_loader, validation_loader = split_dataset(dataset, validation_split, batch_size, shuffle_dataset, random_seed)
dataloaders_dict = {'train': train_loader, 'val': validation_loader}

# Define optimizer
my_optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Setup the loss
my_criterion = nn.MSELoss()

# Train and evaluate
modified_model, hist = train_model(modified_model,
                                   dataloaders_dict,
                                   my_criterion,
                                   my_optimizer,
                                   num_epochs=num_epochs)

----------
Epoch 0/29
----------
train Loss: 59295.5174

val Loss: 31727.9009

Training complete in 0m 5s
Best val loss: 31727.900949

----------
Epoch 1/29
----------
train Loss: 39744.6625

val Loss: 33221.7919

Training complete in 0m 10s
Best val loss: 31727.900949

----------
Epoch 2/29
----------
train Loss: 30862.4676

val Loss: 31988.4576

Training complete in 0m 15s
Best val loss: 31727.900949

----------
Epoch 3/29
----------
train Loss: 30938.8482

val Loss: 27628.3767

Training complete in 0m 21s
Best val loss: 27628.376674

----------
Epoch 4/29
----------
train Loss: 28141.9481

val Loss: 28360.0926

Training complete in 0m 26s
Best val loss: 27628.376674

----------
Epoch 5/29
----------
train Loss: 27811.1186

val Loss: 27137.9146

Training complete in 0m 31s
Best val loss: 27137.914621

----------
Epoch 6/29
----------
train Loss: 27508.7069

val Loss: 26963.5698

Training complete in 0m 36s
Best val loss: 26963.569754

----------
Epoch 7/29
----------
train Loss: 2712

In [47]:
for name, param in modified_model.named_parameters():
  if (name == 'classifier.4.weight') or (name == 'classifier.4.bias'):
    print(name, param)

classifier.4.weight Parameter containing:
tensor([[[[ 8.0498e+00]],

         [[ 2.7167e+00]],

         [[ 1.8490e+00]],

         ...,

         [[ 4.6232e+00]],

         [[ 2.0870e+00]],

         [[-2.7501e-01]]],


        [[[-3.2070e-02]],

         [[-3.3037e-03]],

         [[-2.2896e-02]],

         ...,

         [[-1.8999e-02]],

         [[-3.9636e-02]],

         [[-1.4570e-02]]]], device='cuda:0', requires_grad=True)
classifier.4.bias Parameter containing:
tensor([30.1415,  0.0376], device='cuda:0', requires_grad=True)
